In [ ]:
#for SoccerPitch and more analysis pitch visualization library 
pip install mplsoccer

In [ ]:
from mplsoccer.pitch import Pitch
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
fig, ax = pitch.draw()

In [ ]:
from mplsoccer.pitch import Pitch
pitch = Pitch(pitch_type='statsbomb', line_color='black', stripe=True)
fig, ax = pitch.draw()

In [ ]:
# Create a matplotlib figure with desired size
fig, ax = plt.subplots(figsize=(12, 8))

# Create the Pitch object and draw the pitch on the specified figure and axis
pitch = Pitch(pitch_type='statsbomb', line_color='black')
pitch.draw(ax=ax)

# Add your soccer elements here (e.g., scatter, lines, text, etc.)

plt.show()  # This displays the plot in the notebook

# Save the figure as a PNG file with specified DPI
fig.savefig('soccer_pitch.png', dpi=300)  # Adjust filename and DPI as needed


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch,VerticalPitch
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup
import json
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [ ]:
url=input("enter the url:" " ")

In [ ]:
response = requests.get(
    url,
    headers={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"} # you'll be blocked if you don't use some type of user agent
)

In [ ]:
response.status_code

In [ ]:
soup=BeautifulSoup(response.content,"lxml")
scripts=soup.find_all("script")

In [ ]:
scripts

In [ ]:
strings=scripts[22].string #change scripts try range 15-25 to find as per whoScored page script by search matchCentreData 
strings

In [ ]:
ind_start=strings.index("matchCentreData:")+17
ind_end=strings.index(",\r\n            matchCentreEventTypeJson")

In [ ]:
jason_data=strings[ind_start:ind_end]

In [ ]:
data=json.loads(jason_data)

In [ ]:
events=data["events"]

In [ ]:
df=pd.DataFrame(events)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
# create function for cleaning
def clean_data(x):
    x1=x.apply(lambda x:str(x))
    x2=x1.str.split(":",n=3,expand=True)[2]
    x3=x2.str.replace("[^a-zA-Z0-9]",'',regex=True)
    return x3

In [ ]:
# for period ,type
df["period"]=clean_data(df["period"])
df["type"]=clean_data(df["type"])
df["outcomeType"]=clean_data(df["outcomeType"])

In [ ]:
df1=df[["id","eventId","minute","second","teamId","x","y","endX","endY","period","type","outcomeType","playerId"]]

In [ ]:
df

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('atlanta-liverpool_3-0_passxy.csv', index=False)

In [ ]:
# Create new columns for recipient
df["passer_ID"] = df["playerId"]
df["recipient_ID"] = df["playerId"].shift(-1)

In [ ]:
df.head()

In [ ]:
from google.colab import files

file_name_PasRecpt = input("Enter the file name you want to use (without the .xlsx extension): ")

# Save the selected columns to an Excel file
selected_columns.to_excel(file_name_PasRecpt + '.xlsx', index=False)

# Download the Excel file
files.download(file_name_PasRecpt + '.xlsx')

In [ ]:
#FETCH & MERGE PLAYER NAME WITH PLAYER ID

player_data=data["playerIdNameDictionary"]

In [ ]:
player_data=pd.Series(player_data)
player_data=pd.DataFrame(player_data)
player_data.reset_index(inplace=True)

In [ ]:
player_data.columns=["playerId","playername"]

In [ ]:
player_data["playerId"]=player_data["playerId"].apply(lambda x:int(x))

In [ ]:
#MARGE THE NAME OF THE PLAYER AND EVENT DATA
df2=pd.merge(df1,player_data,on="playerId",how="left")

In [ ]:
df2